**Activate google colab**


In [207]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [208]:
pip install ckip-transformers

**import module**

In [209]:
# old  module
import numpy as np
import tensorflow as tf  
from tensorflow import keras
from tensorflow.keras import layers 

#new  module
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import Transformer
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import torch.optim as optim
import random

random.seed(42)

**read Data**

In [210]:
def read_data(file):
    #load and clean'\n' 
    with open(file,'r') as ff:
        test = ff.readlines()
    test = [i.replace('\n','') for i in test]
    test = [i.split(' ') for i in test]
    
    #reshape data
    text_final = []
    BIO_final = [] 
    text_tmp = []
    BIO_tmp = []
    for i in test:
        if len(i) > 1:
            text_tmp.extend([i[0]])
            BIO_tmp.append(i[1])
        elif len(i) == 1:
            text_final.append(text_tmp)
            BIO_final.append(BIO_tmp)
            text_tmp = []
            BIO_tmp = []       
    cls_value = [i.pop(0) for i in BIO_final]
    [i.pop(0) for i in text_final]
    return text_final, BIO_final, cls_value

def label2index(label,label_dict):
    index_label = []
    for i in label:
        lab_tmp = [label_dict[j]for j in i] 
        index_label.append(lab_tmp)
    return index_label

**get representation**

In [211]:
def get_representation(output):
    # shape: (seq_len, vocab_size)
    hidden_states = output[1]

    token_embeddings = torch.stack(hidden_states, dim=0)
    # remove dimension 1 (batches)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # swap dimension 0 and 1
    token_embeddings = token_embeddings.permute(1, 0, 2)
    # the last hidden layer output (2+seq_len, 768)
    hidden_states = [token[-1] for token in token_embeddings]

    return hidden_states

**decoder for ID**

In [212]:
# decoder for ID
class ID_module(nn.Module):
    def __init__(self):
        super(ID_module, self).__init__()
        self.linear = nn.Linear(768, 2)
    
    def forward(self, X):
        X = self.linear(X)
        return X

**decoder for SF**

In [213]:
# decoder for SF
class SF_module(nn.Module):
    def __init__(self):
        super(SF_module, self).__init__()
        self.linear = nn.Linear(768, n_classes)
    
    def forward(self, X):
        X = self.linear(X)
        return X

**Train ID**

In [214]:
# dataset: [[sen_1, 1], [sen_2, 1], [sen_3, 2], ...]

def train_ID(ID_model, dataset, epochs, lr):
    ID_model.train()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(ID_model.parameters(), lr=lr, momentum=0.9)

    for epoch in range(epochs):
        totalLoss = 0
        accuracy = 0
        count = 0

        for X, y in dataset:
            y = torch.LongTensor([int(y)]).to(DEVICE)
            optimizer.zero_grad()

            X = " ".join(X)
            X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
            X_ids = X_encoding['input_ids'].to(DEVICE)

            with torch.no_grad():
                output = model(X_ids)
                
            # get the 768-d representation of [CLS]
            ID_input = get_representation(output)[0]
            ID_output = ID_model(ID_input).unsqueeze(0)

            ##### record the loss? #####
            _, predicted = torch.max(ID_output.data, 1)
            count += len(X_ids) - 2
            accuracy += (predicted == y).sum().item()

            loss = criterion(ID_output, y)

            totalLoss += loss.item()
            # print("Loss: {}".format(totalLoss))
            
            loss.backward()
            optimizer.step()

        print("Train Loss: {}".format(totalLoss / count))
        print("Train Accuracy: {}".format(accuracy / count))

    return ID_model

**Train SF**

In [215]:
# dataset: [[sen_1, [1,1,1,2,3,...]], [sen_2, [1,1,2,3,...]], [sen_3, [1,1,1,2,...]], ...]

def train_SF(SF_model, dataset, epochs, lr):
    SF_model.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(SF_model.parameters(), lr=lr, momentum=0.9)

    for epoch in range(epochs):
        totalLoss = 0
        count = 0
        y_true = []
        y_pred = []

        for X, y in dataset:
            y_true.extend(y)
            optimizer.zero_grad()

            X = " ".join(X)
            X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
            X_ids = X_encoding['input_ids'].to(DEVICE)

            with torch.no_grad():
                output = model(X_ids)

            # get the 768-d representation of other tokens than [CLS]
            SF_input = get_representation(output)[1:-1]
            # for each position, predict a label with SF_model and back propagate the loss
            for p in range(len(X_ids[0]) - 2):
                y_single = torch.LongTensor([y[p]]).to(DEVICE)
                SF_output = SF_model(SF_input[p]).unsqueeze(0)

                # print(y_single)

                y_pred.append(torch.argmax(SF_output).item())

                # record the loss
                loss = criterion(SF_output, y_single)
                # print("Loss: {:2f}".format(loss))
                count += 1
                totalLoss += loss.item()*5
    
                loss.backward()
                optimizer.step()

        print("Train Loss: {:2f}".format(totalLoss / count))
        # print("Train Loss: {}".format(totalLoss))
        # print f1-score
        print( "F1 score: {:2f}".format(f1_score(y_true, y_pred, average = 'macro')) )

    return SF_model

**load ckip BERT**

In [216]:
from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForTokenClassification,
)

DEVICE = torch.device('cpu')

# language model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
model = AutoModelForMaskedLM.from_pretrained('ckiplab/bert-base-chinese', output_hidden_states=True).to(DEVICE)

**id_label_translation**

In [217]:
n_classes = 5
intent_to_id = {
    "income": 0,
    "expense": 1
}
slot_to_id = {
    "O": 1,
    "B-item": 2,
    "I-item": 3,
    "B-money": 4,
    "I-money": 5
}

id_to_intent = {v: k for (k, v) in intent_to_id.items()}
id_to_slot = {v: k for (k, v) in slot_to_id.items()}



**load train data**

In [218]:
data_file = "/content/gdrive/MyDrive/NLP/final_project/final_dataset/train_data_income99_expense_99.txt"
text_final, BIO_final, cls_value = read_data(data_file)

##shuffle data set
random.seed(42)
ID_dataset = [z for z in zip(text_final, cls_value)]
random.shuffle(ID_dataset)
random.seed(42)

BIO_final_id = label2index(BIO_final, slot_to_id)
SF_dataset = [z for z in zip(text_final, BIO_final_id)]
random.shuffle(SF_dataset)

## get bert representation
x_bert = []
y_true = []
for X, y in SF_dataset:
  y_true.append(y)

  X = " ".join(X)
  X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
  X_ids = X_encoding['input_ids'].to(DEVICE)
  with torch.no_grad():
      output = model(X_ids)
  # get the 768-d representation of other tokens than [CLS]
  SF_input = get_representation(output)[1:-1]
  SF_input = [np.array(i) for i in SF_input]
  x_bert.append(SF_input)

## pading for X and Y
for i in x_bert:
  while len(i) < 18:
    i.append(np.zeros(768))
for i in y_true:
  while len(i) < 18:
    i.append(0)

**load valid data**

In [219]:
data_file = "/content/gdrive/MyDrive/NLP/final_project/final_dataset/valid_data_income31_expense40.txt"
text_final, BIO_final, cls_value = read_data(data_file)

##shuffle data set
random.seed(42)
ID_dataset = [z for z in zip(text_final, cls_value)]
random.shuffle(ID_dataset)
random.seed(42)

BIO_final_id = label2index(BIO_final, slot_to_id)
SF_dataset = [z for z in zip(text_final, BIO_final_id)]
random.shuffle(SF_dataset)

## get bert representation
x_bert_valid = []
y_true_valid = []
for X, y in SF_dataset:
  y_true_valid.append(y)

  X = " ".join(X)
  X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
  X_ids = X_encoding['input_ids'].to(DEVICE)
  with torch.no_grad():
      output = model(X_ids)
  # get the 768-d representation of other tokens than [CLS]
  SF_input = get_representation(output)[1:-1]
  SF_input = [np.array(i) for i in SF_input]
  x_bert_valid.append(SF_input)

## pading for X and Y
for i in x_bert_valid:
  while len(i) < 20:
    i.append(np.zeros(768))
for i in y_true_valid:
  while len(i) < 20:
    i.append(0)

In [220]:
##change format
x_bert = [np.array(i) for i in x_bert]
y_true = [np.array(i) for i in y_true]
x_bert_valid = [np.array(i) for i in x_bert_valid]
y_true_valid = [np.array(i) for i in y_true_valid]

x_bert = np.array(x_bert)
y_true = np.array(y_true)
x_bert_valid = np.array(x_bert_valid)
y_true_valid = np.array(y_true_valid)


print(len(x_bert))
print(len(y_true))
print(len(x_bert_valid))
print(len(y_true_valid))



198
198
71
71


**bidirectional LSTM(max sequence length 18)**

In [ ]:
#set parameter
samples, timesteps, features = len(x_bert), len(x_bert[0]), len(x_bert[0][0])

network = keras.Sequential()

#masking layer
network.add(keras.layers.Masking(mask_value=0.,
                                  input_shape=(timesteps, features)))
#model.LSTM
forward_layer = layers.LSTM(32, return_sequences=True)
# backward_layer = layers.LSTM(32, return_sequences=True, go_backwards=True)
# network.add(layers.Bidirectional(forward_layer, backward_layer=backward_layer))
network.add(forward_layer)
network.add(layers.Dropout(0.1))
# network.add(layers.LSTM(64,return_sequences = 'True'))

#output layer
network.add(layers.Dense(6,activation='softmax'))

#error
network.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'RMSprop',metrics = ['accuracy'])

# checkpoint
filepath="/content/gdrive/MyDrive/NLP/final_project/train_record/weights-improvement-{epoch:02d}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only  = False,save_weights_only = False,save_freq="epoch")
callbacks_list = [checkpoint]
#
train_history = network.fit(x = x_bert,y = y_true,validation_data=(x_bert_valid, y_true_valid), epochs = 10, batch_size = 1, callbacks = callbacks_list, verbose = 2)
network.summary()
network.save('/content/gdrive/MyDrive/NLP/final_project')

Epoch 1/10


198/198 - 6s - loss: 0.2043 - accuracy: 0.9041 - val_loss: 0.0993 - val_accuracy: 0.9553

Epoch 00001: saving model to /content/gdrive/MyDrive/NLP/final_project/train_record/weights-improvement-01.hdf5
Epoch 2/10
198/198 - 2s - loss: 0.0558 - accuracy: 0.9788 - val_loss: 0.0830 - val_accuracy: 0.9591

Epoch 00002: saving model to /content/gdrive/MyDrive/NLP/final_project/train_record/weights-improvement-02.hdf5
Epoch 3/10
198/198 - 2s - loss: 0.0285 - accuracy: 0.9894 - val_loss: 0.0909 - val_accuracy: 0.9617

Epoch 00003: saving model to /content/gdrive/MyDrive/NLP/final_project/train_record/weights-improvement-03.hdf5
Epoch 4/10
198/198 - 2s - loss: 0.0133 - accuracy: 0.9940 - val_loss: 0.1014 - val_accuracy: 0.9617

Epoch 00004: saving model to /content/gdrive/MyDrive/NLP/final_project/train_record/weights-improvement-04.hdf5
Epoch 5/10
198/198 - 2s - loss: 0.0066 - accuracy: 0.9968 - val_loss: 0.1092 - val_accuracy: 0.9591

Epoch 00005: saving model to /content/gdrive/MyDrive/NLP/f

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/NLP/final_project/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/NLP/final_project/assets


**PREDICT**

In [201]:
data_file = "/content/gdrive/MyDrive/NLP/final_project/final_dataset/valid_data_income31_expense40.txt"
text_final, BIO_final, cls_value = read_data(data_file)

##shuffle data set
random.seed(42)
ID_dataset = [z for z in zip(text_final, cls_value)]
random.shuffle(ID_dataset)
random.seed(42)

BIO_final_id = label2index(BIO_final, slot_to_id)
SF_dataset = [z for z in zip(text_final, BIO_final_id)]
random.shuffle(SF_dataset)

## get bert representation
x_bert_valid = []
y_true_valid = []
for X, y in SF_dataset:
  y_true_valid.append(y)

  X = " ".join(X)
  X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
  X_ids = X_encoding['input_ids'].to(DEVICE)
  with torch.no_grad():
      output = model(X_ids)
  # get the 768-d representation of other tokens than [CLS]
  SF_input = get_representation(output)[1:-1]
  SF_input = [np.array(i) for i in SF_input]
  x_bert_valid.append(SF_input)

## predict
y_pred = []
for i in x_bert_valid:
  tt = []
  tt.append(i)
  output_label = network.predict_classes(np.array(tt))
  y_pred.append(output_label)

print(len(y_pred))
##handinput
# X = '設計一款遊戲賺了1043043元'
# X = " ".join(X)
# X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
# X_ids = X_encoding['input_ids'].to(DEVICE)
# with torch.no_grad():
#     output = model(X_ids)
# # get the 768-d representation of other tokens than [CLS]
# LSTM_input = get_representation(output)[1:-1]
# LSTM_input = [np.array(i) for i in LSTM_input]
# tt=[]
# tt.append(np.array(LSTM_input))
# output_label = network.predict_classes(np.array(tt))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


71


In [165]:
#caluculate F1 score
from sklearn.metrics import f1_score, recall_score, precision_score


(1, 13, 768)


In [205]:
print(y_pred[3])
print(SF_dataset[3][0])
print(y_true_valid)


[[1 1 1 1 1 1 4 5 5 5 5 5 5 5 5 5 5 5 5 5]]
['公', '司', '今', '年', '賺', '了', '1', '2', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '元']


In [200]:
#handinput
X = '設計一款遊戲賺了1043043元'
X = " ".join(X)
X_encoding = tokenizer.encode_plus(X, add_special_tokens=True, return_tensors='pt')
X_ids = X_encoding['input_ids'].to(DEVICE)
with torch.no_grad():
    output = model(X_ids)
# get the 768-d representation of other tokens than [CLS]
LSTM_input = get_representation(output)[1:-1]
LSTM_input = [np.array(i) for i in LSTM_input]
# tt=[]
# tt.append(np.array(LSTM_input))
output_label = network.predict_classes(np.array(LSTM_input))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


ValueError: ignored